In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
from time import sleep
import pandas as pd

In [ ]:
def extract_claim_list(company, ini_page, end_page):
    # Selenium configurations
    options = webdriver.ChromeOptions()
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    options.page_load_strategy = 'eager'
    driver = webdriver.Chrome(options=options)

    base_url = "https://www.reclameaqui.com.br/empresa/" + company
    title_link = []

    # Extract data
    for page_number in range(ini_page, end_page+1):
        url = base_url + "/lista-reclamacoes/?pagina=" + str(page_number)
        driver.get(url)
        sleep(3)
        page_html = bs(driver.page_source, "html.parser")
        claims_html = page_html.find_all(class_='sc-1pe7b5t-0 iQGzPh')

        for element in claims_html:
            element_title_link = [element.find('h4')['title'], element.find('a')['href']]
            title_link.append(element_title_link)
    
    driver.quit()
    return title_link

In [ ]:
# Extract data
title_link = extract_claim_list('claro', 1, 50)

In [ ]:
# Save into csv file
main_page = pd.DataFrame(title_link, columns=['Title', 'link'])
main_page.to_csv('reclame_aqui_main.csv')
print(main_page.shape)

In [ ]:
# Open list of claims
main_page = pd.read_csv('reclame_aqui_main.csv')
main_page.head()

In [ ]:
# Get descriptions
options = webdriver.ChromeOptions()
options.add_experimental_option('excludeSwitches', ['enable-logging'])
options.page_load_strategy = 'eager'
driver = webdriver.Chrome(options=options)

all_descriptions = []

for index, url in enumerate(main_page['link']):
    # Reset browser each 50 requests
    if index%50 == 0 and index!=0:
        print(f"Sample: {index}")
        driver.quit()
        driver = webdriver.Chrome(options=options)
        driver.get(full_url)
    
    full_url = "https://www.reclameaqui.com.br" + url
    driver.get(full_url)
    sleep(1.5)
    page_html = bs(driver.page_source, "html.parser")
    try:
        description = page_html.find('p', {'data-testid': 'complaint-description'}).get_text()
    except:
        description = ""
    all_descriptions.append(description)
driver.quit()

In [ ]:
# Add descriptions to dataframe and save new csv
claims_full = main_page.copy()
claims_full['description'] = all_descriptions
claims_full.to_csv('reclame_aqui_full.csv')
claims_full.head()